# DEAGO tutorial

## Introduction

In this tutorial, we use DEAGO to generate user-friendly analysis reports for an RNA-Seq dataset.

The following software versions were used when preparing this tutorial:

* Bio-Deago 1.2.0

* deago 1.1.2

The dataset we will be using forms part of the following publication:

> **Epithelial IL-22RA1-Mediated Fucosylation Promotes Intestinal Colonization Resistance to an Opportunistic Pathogen**  
> Pham TA, Clare S, Goulding D, Arasteh JM _et al._  
> _Cell Host Microbe. 2014 Oct 8;16(4):504-16. doi: 10.1016/j.chom.2014.08.017._    
> PMID: [25263220](https://www.ncbi.nlm.nih.gov/pubmed/25263220)

For Sanger pathogen users, you can find this dataset under study id **2319** using the **`pf`** commands.  Click [here](http://mediawiki.internal.sanger.ac.uk/index.php/Pathogen_Informatics_Command_Line_Scripts) for more information.

***

## Overview

This tutorial is comprised of the following sections:

* [Tutorial dataset](#Tutorial-dataset)
* [Input data](#Input-data)
  * [Count data](#Count-data)
  * [Sample/condition mappings](#Sample/condition-mappings)
  * [Annotation file](#Annotation-file)
* [QC analysis](#QC-analysis)
  * [QC output files](#QC-output-files)
  * [QC analysis report](#QC-analysis-report)
  * [Presentation quality images](#Presentation-quality-images)
* [DE analysis](#DE-analysis)
  * [DE analysis report](#DE-analysis-report)
  * [DE results files](#DE-results-files)
* [GO term enrichment analyses](#GO-term-enrichment-analyses)
  * [GO enrichment analysis report](#GO-enrichment-analysis-report)
  * [GO enrichment results files](#GO-enrichment-results-files)
* [Troubleshooting](#Troubleshooting)

***

## Tutorial dataset

**This tutorial assumes you have DEAGO installed and available in your PATH.**  

The tutorial dataset is in the directory called [**deago_tutorial**](deago_tutorial) which is part of this repository and contains everything you need to begin the tutorial. 

Move into the tutorial directory:

In [ ]:
    cd deago_tutorial

And now we're ready to begin!

***

## Input data

We'll be using three types of input data:

* **`counts`** - a directory containing count data files (one file per sample)

* **`targets.txt`** - a sample/condition mapping file 

* **`ensembl_mm10_deago_formatted.tsv`** - an annotation file containing gene symbols and GO terms

For more information on the input datatypes DEAGO uses and their formats see [Input files](Input-files.ipynb).

### Count data

DEAGO looks in a single folder for all of your count data, one file per sample. We will need to tell DEAGO the location or path for this folder.

Let's take a look at our tutorial count data which can be found in the **counts** directory:

In [ ]:
    ls counts

You should see a list of 32 count files were generated using [featureCounts](http://bioinf.wehi.edu.au/featureCounts/). Let's take a look inside one of the count files:

In [ ]:
    head counts/8380_3#1.390176.pe.markdup.bam.featurecounts.csv

You should see:

    # Program:featureCounts v1.4.5-p1; Command:"featureCounts" "-O" "-T" "1" "-t" "exon" "-g" "gene_id" "-a" 
    "/lustre/scratch118/infgen/pathogen/pathpipe/refs/Mus/musculus/Mus_musculus_mm10.gtf" "-o"
    "390176.pe.markdup.bam.featurecounts.csv" "390176.pe.markdup.bam"
    Geneid  Chr     Start   End     Strand  Length  390176.pe.markdup.bam
    ENSMUSG00000090025      1       3054233 3054733 +       501     0
    ENSMUSG00000064842      1       3102016 3102125 +       110     0
    ...

There are several things to notice about the featurecounts file format:

* Gene identifiers are in the first column **`Geneid`** (e.g. ENSMUSG00000090025)


* Gene counts are in the last column (**`7`**) 


* The first line of the file is a comment which gives the details of the program and command used to generate the count file and the fields are tab-delimited (**`\t`**)

DEAGO has an option called **`count_type`** which we will be using in our analyses. Adding **`--count_type featurecounts`** to our commands will tell DEAGO to expect count files which were generated by featureCounts and have the above characteristics. 

### Sample/condition mappings

DEAGO also requires a targets file which tells it which counts file is associated with each sample and the experimental conditions that were applied.

Let's take a look at an example targets file called [**`targets.txt`**](deago_tutorial/targets.txt):

In [ ]:
    head targets.txt

In our targets file, each row corresponds to a sample. There are three columns which DEAGO always expects to find in a targets file:

* **`filename`** - name of the sample count file in the counts directory


* **`condition`** - experimental condition that was applied


* **`replicate`** - number or phrase representing a replicate group

However, notice that in addition to the three expected columns we also have **`cell_type`** and **`treatment`**. That's because the tutorial dataset had two factors, cell type (**WT**/**KO**) and treatment (**Ctrl**/**IL22**). 

    condition	cell_type	treatment	replicate	filename
    WT_Ctrl	WT	Ctrl	2.1	8380_3#1.390176.pe.markdup.bam.featurecounts.csv
    WT_Ctrl	WT	Ctrl	2.2	8380_3#2.390269.pe.markdup.bam.featurecounts.csv
    WT_IL22	WT	IL22	2.2	8380_3#4.389017.pe.markdup.bam.featurecounts.csv

This dataset also has 4 **biological replicates** and 2 **technical replicates** for each condition represented in the `replicate` column.  For example, replicate 1.2 is in biological replicate group 1 and is the second technical replicate for that sample.

### Annotation file

Because we want to see the gene symbols that are linked to each gene identifier and to perform GO enrichment analyses we also need an annotation file. You should see two annotation files in your tutorial directory:

* [**`ensembl_mm10.tsv`**](deago_tutorial/ensembl_mm10.tsv) - this is the same as *mart_export.txt* which was generated in the [Preparing an annotation file](Preparing-an-annotation-file.ipynb) section

* [**`ensembl_mm10_deago_formatted.tsv`**](deago_tutorial/ensembl_mm10_deago_formatted.tsv) - a DEAGO-formatted version of [**`ensembl_mm10.tsv`**](deago_tutorial/ensembl_mm10.tsv)

You can convert [**`ensembl_mm10.tsv`**](deago_tutorial/ensembl_mm10.tsv) into a DEAGO-formatted annotation file using:

In [ ]:
    mart_to_deago -a ensembl_mm10.tsv

This will generate a file called **`deago_annotation.tsv`** which is identical to [**`ensembl_mm10_deago_formatted.tsv`**](deago_tutorial/ensembl_mm10_deago_formatted.tsv).  If you wanted to generate a file with that name you could have used the **`-o`** option to define the output filename:

In [ ]:
    mart_to_deago -a ensembl_mm10.tsv -o ensembl_mm10_deago_formatted.tsv

For more information on the input file formats or preparing an annotation see [Input files](Input-files.ipynb) and [Preparing an annotation file](Preparing-an-annotation-file.ipynb).

Now we've got all our input data ready, let's start analysing it!

***

## QC analysis

Each DEAGO analysis should be self-contained, so let's create a new directory for our QC analysis:

In [ ]:
    mkdir qc && cd qc

To run our QC analysis:

In [ ]:
    deago --build_config -c ../counts -t ../targets.txt --count_type featurecounts --qc

Here is a brief explanation of the options we used and what they mean:

* **`--build_config`** tells DEAGO that we want to build a new config file using the command line parameters 


* **`-c`** tells DEAGO the location of the folder containing our count files


* **`-t`** tells DEAGO the location of our sample/condition mapping file


* **`--count_type`** tells DEAGO the format of the count data (e.g. featurecounts) setting the values for **`count_column`**, **`skip_lines`**, **`gene_ids`**, **`count_delim`**.


* **`--qc`** tells DEAGO to only run the QC analysis

You can always find out more information about the options available with DEAGO by running:

In [ ]:
    deago -h

### QC output files

Once your analysis has finished, you should see several new files:

* **`deago.config`** - config file with key/value parameters defining the analysis  


* **`deago.rlog`** - log of the R output generated when converting the R markdown to HTML  


* **`deago_markdown.Rmd`** - R markdown used to run the analysis  


* **`deago_markdown.html`** - HTML report generated from the R markdown   

### QC analysis report

The one we are interested in is **`deago_markdown.html`** which is your QC analysis report.  Go ahead and open it in a web browser (e.g. Chrome, Firefox, IE, Safari...).

The report has a sidebar on the left so you can quickly navigate the report.

![Navigation panel](images/deago_navigation.png)

Click on **`Pipeline configuration`**.  Here you will see that the report contains the commands used to run the analysis (grey boxes) and their output. The **`Pipeline configuration`** section shows the parameters that were used for the analysis. This can be useful for finding input/output data, troubleshooting and debugging.

![Pipeline configuration](images/deago_parameters.png)

Let's take a quick look at some of the QC plots. Click on **`QC Plots`** in the left-hand panel.

First, check to see if there were any problems with the sequencing. The _Total read counts per sample_ plot does what it says on the tin and shows us the number of reads overlapping features in each sample. If one sample has a particularly low count compared to the others, it may indicate an issue and so you should take a closer look at that sample in some of the other QC plots.

![Total read counts per sample plot](images/reads_per_sample.png)

The principal component analysis (PCA) and sample-to-sample distances plot are indicators of variation and will show how well the samples cluster together. These are a quick way of spotting outliers and potential batch effects.

In the PCA plot, samples are coloured by condition and the sample labels are a combination of the sample condition and the replicate number.  Here we can see our samples cluster reasonably well with distinct groups for **wt_ctrl**, **wt_il22** and **ko_ctrl**/**ko_il22**.

![PCA plot](images/pca.png)

To compliment the PCA plot, we also have a scree plot which is a histogram of the percentage contribution of each of the principal components (PC). The points indicate the cumulative total and the lines represent broad cutoffs of 70% and 90%.  Ideally, in a two-factor analysis we'd hope to see the cumulative total of PC1 and PC2 reaching 70%, although this may not always be the case.

![PCA scree plot](images/pca_scree.png)

There is also a sample-to-sample distances plot shaded by the distance (or variability) between samples. The darker the box, the more similar the samples (or the smaller the distance between them).  We can see that the samples clearly form three distinct groups: **wt_ctrl**, **wt_il22** and **ko** samples.

![Sample distances plot](images/sample_distances.png)

### Presentation quality images

The images in the report are small to make the report easy to share and so may not be ideal for presentations, posters or publications. To generate better quality images we can use the **`--keep_images`** option which will include an **`images`** folder to your timestamped results directory containing one file per plot.

First, create a new analysis directory:

In [ ]:
    cd .. && mkdir qc_with_images && cd qc_with_images
    deago --build_config -c ../counts -t ../targets.txt --count_type featurecounts --qc --keep_images

Now, let's try to identify some differentially expressed (DE) genes.

***

## DE analysis

First, create a new analysis directory:

In [ ]:
    cd .. && mkdir de_analysis && cd de_analysis

And then run your DE analysis:

In [ ]:
    deago --build_config -c ../counts -t ../targets.txt --count_type featurecounts --control WT_Ctrl 

Notice that we have used a new option **`--control`** which tells DEAGO the condition you want to use as your reference or control, in this case **WT_Ctrl**. We use **`--control`** to define our reference condition because, by default, R chooses the reference condition based on alphabetical order. It would assume that from our four conditions (**KO_Ctrl**, **KO_IL22**, **WT_Ctrl** and **WT_IL22**) that KO_Ctrl is our reference condition because it is first alphabetically.  The value you use **must** be in the condition column in your targets file and is _case insensitive_.

This also makes an assumption that the FDR cutoff (alpha) will be **0.05** (default). If you are expecting to use a different cutoff in your downstream filtering, use the **`-q`** option to define the FDR cutoff (e.g. -q 0.01).

### DE analysis report

Open **`deago_markdown.html`** in a web browser. In addition to the QC sections we saw before, you should now see a new option in the left-hand sidebar called **`Pairwise contrasts`**. Click on it and it will take you to your DE analysis results.

First there is a **`Contrast summary`** section which contains a summary table showing how many genes are up-regulated or down-regulated in each contrast (comparison between two sample groups). We can see that there were no differentially expressed (DE) genes between the knockout (KO) samples induced with IL22 (ko_il22) and the control knockout samples. However, there were 860 DE genes between wildtype (WT) and knockout (KO) samples induced with IL22, 510 up-regulated in the WT samples compared to the KO samples and 350 down-regulated.

![DE summary table](images/DEsummary.png)

If there are 2-4 contrasts in the analysis, there will also be a Venn diagram showing the overlap/differences in total DE genes between contrasts. We have 6 contrasts in this analysis, so no Venn diagram was generated, but an example would be:

![DE Venn diagram](images/DEvenn.png)

The DE analysis report then has a series of subsections, one per contrast. Each contrast section has an MA plot and a volcano plot. The top 5 up- and down-regulated gene identifiers are labelled on plots. If an annotation with gene symbols was used (see [GO analysis](#GO-term-enrichment-analyses)) then the point labels will be the gene symbols and not the gene identifiers.

![DE volcano and MA plots](images/DEvolcanoMA.png)

Each contrast section will also have a DE results table which contains genes with an adjusted p-value < 0.01 and a log2 fold change >= 2 or <= -2. This is to reduce the number of genes in the table so that the HTML report is compact and sharable.

![DE contrast results table](images/DEtable.png)

These tables contain the DESeq2 results and are where you will find your adjusted p-values and log2 fold change values. Also, as our gene identifiers are Ensembl identifiers, they have been converted to a link and if you click on one, it will take you to the current Ensembl page for that gene stable ID. In this example, we had include an annotation in the analysis and so the gene symbols are also shown.

All of the tables are interactive and can be searched or filtered. The paper describes the up-regulation of _Fut2_ by IL-22RA1 signalling, so let's take a look. The search box at the top right searches the whole table, so we can use it to search for any _Fut_ genes.

![Searching DE tables](images/DEsearch.png)

This gives us two genes: _Fut2_ and _Fut9_.

Now, say we wanted to only see the up-regulated _Fut_ genes.  We can limit searches and filters to a single column by using the search/filter boxes at the top of each column. Use the selector at the top of the **`log2FoldChange`** column to only include values greater than 0 (i.e. drag the left selector). 

![Searching DE tables](images/DEfilter.png)

We are now left with the only up-regulated _Fut_ gene, _Fut2_.

### DE results files

The report tables are limited to genes with an adjusted p-value < 0.01 and a log2 fold change >= 2 or <= -2. However, you are likely to want to explore and filter these results using different thresholds. So, DEAGO also writes the unfiltered results table containing all genes to individual files, one per contrast in your timestamped results directory.

For more information on these output files, their formats and naming convention see [Output files](Output-files.ipynb).

***

## GO term enrichment analyses

First, create a new analysis directory:

In [ ]:
    cd .. && mkdir go_analysis && cd go_analysis

And the run your GO term enrichment analysis:

In [ ]:
    deago --build_config -c ../counts -t ../targets.txt \
                         --count_type featurecounts --control WT_Ctrl \
                         -a ../ensembl_mm10_deago_formatted.tsv --go

We have used two new options here: **`-a`** and **`--go`**. For a GO term enrichment analysis we need to know which GO terms are associated with each gene identifier. The identifier/GOterm/symbol mappings can be found in our annotation file [**`ensembl_mm10_deago_formatted.tsv`**](deago_tutorial/ensembl_mm10_deago_formatted.tsv) (see [Annotation file](#Annotation-file)). We tell DEAGO the location of the annotation file using the **`-a`** option. The **`--go`** option tells DEAGO to perform a GO term enrichment analysis in addition to your QC and DE analyses as it doesn't include this by default.

### GO enrichment analysis report

Open **`deago_markdown.html`** in a web browser. You'll already be familiar with the **`QC plots`** and **`Pairwise contrasts`** sections. Click on **`Pairwise contrasts`** and then go to the contrast subsection for **`wt_il22_vs_ko_il22`**. You should now see six new subsections, *6.7.2 - 6.7.7*, which contain your GO term enrichment analysis results for that contrast.

The first three subsections have the prefix **`GO term enrichment - BP`** and contain the results for the _Biological Processes (BP)_. The remaining three subsections have the prefix **`GO term enrichment - MF`** and contain the results for the _Molecular Functions (MF)_.

For each GO level (BP or MF) there is a subsection containing the results from GO term enrichment analyses which were run using all DE genes, up-regulated genes only and down-regulated genes only.

Let's take a look at the BP (upregulated genes only) table for the contrast wt_il22_vs_ko_il22 (subsection 6.7.3). All of the results tables are interactive. Despite performing a single-factor analyis, we can still find the top GO description for up-regulated genes _inflammatory response_ in our results table (GO:0006954).

![GO term enrichment results table for wt_il22_vs_ko_il22 BP (upregulated genes only)](images/BPupTable.png) 

The paper also mentions _Prdm1_ which is upregulated by IL-22RA1 signalling and is a susceptibility gene in to inflammatory bowel disease. Let's see whether it's associated with any of the enriched GO terms. Type "prdm1" in the top right search box to search the whole table.

![Search for Prdm1 (left)](images/BPupPrdm1Left.png)

We can see that one GO term matches: GO:0031663 (lipopolysaccharide-mediated signaling pathway).

Scrolling to the right, we can see how the search found this match. For each GO term, the DE genes which are associated with that term are also shown in the table.

![Search for Prdm1 (right)](images/BPupPrdm1Right.png)

### GO enrichment results files

DEAGO also writes the GO term enrichment analysis results tables containing the **top 30** significantly enriched GO terms to individual files, split by GO term level (MF and BP), in your timestamped results directory. If possible, there will be three GO tables for each GO term level containing the results for analyses using all genes, up-regulated genes only and down-regulated genes only.

For more information on these output files, their formats and naming convention see [Output files](Output-files.ipynb).

***

## Troubleshooting

If **`deago_markdown.html`** isn't generated, the first thing to do is check the end of the R log file (**`deago.rlog`**). If successful it should be similar to:

    output file: deago_markdown.knit.md
    
    /software/pathogen/external/apps/usr/bin/pandoc +RTS -K512m -RTS deago_markdown.utf8.md --to html --from
    markdown+autolink_bare_uris+ascii_identifiers+tex_math_single_backslash --output /lustre/scratch118/infgen/pathdev
    /vo1/deago_tutorial_test/deago_analysis/deago_markdown.html --smart --email-obfuscation none --self-contained
    --standalone --section-divs --table-of-contents --toc-depth 3 --variable toc_float=1 --variable
    toc_selectors=h1,h2,h3 --variable toc_collapsed=1 --variable toc_smooth_scroll=1 --variable toc_print=1 
    --template /software/pathogen /external/lib/R-3.4/rmarkdown/rmd/h/default.html --no-highlight 
    --variable highlightjs=1 --number-sections --variable 'theme:paper' --include-in-header 
    /tmp/RtmpyTxHuS/rmarkdown-str1782209e9bc2.html --mathjax --variable
    'mathjax-url:https://mathjax.rstudio.com/latest/MathJax.js?config=TeX-AMS-MML_HTMLorMML' 
    
    In addition: Warning messages:
    1: Removed 778 rows containing non-finite values (stat_density). 
    2: Removed 778 rows containing non-finite values (stat_density).

If not, it will show the error which stopped the report being generated.  For example:

    Quitting from lines 206-208 (deago_markdown.Rmd) 
    Error in .local(.Object, ...) : allGenes must be a factor with 2 levels
    Calls: <Anonymous> ... prepareGOdata -> new -> initialize -> initialize -> .local
    In addition: Warning messages:
    1: Removed 778 rows containing non-finite values (stat_density). 
    2: Removed 778 rows containing non-finite values (stat_density). 
    
    Execution halted

This suggests you should look at what's on lines 206-208 in **`deago_markdown.Rmd`** and try to work out the cause of the error.  Once you've fixed it, you can use **`deago_markdown_to_html`** to try and convert your modified markdown file to a new HTML report.  If this doesn't work, you can always try running the commands from the markdown file in R manually to debug the issue.

If this doesn't work and you think there might be bug, please email [path-help@sanger.ac.uk ](mailto:path-help@sanger.ac.uk) attaching your R markdown file, R log file and config file.

[Return to the index](index.ipynb)